In [ ]:
! pip install numpy scipy matplotlib ipywidgets 
! git clone https://github.com/ironcevic/integration_and_kinetics.git
%cd integration_and_kinetics

In [12]:
# 1 IMPORT THE LIBRARIES AND THE DATA
from functions import *

# HERE WE LOAD THE DATA 
raw_data = np.loadtxt('integration_data.csv', delimiter=',', skiprows=1).T
x = raw_data[0]
signal1 = raw_data[1]
signal2 = raw_data[2]
signal3 = raw_data[3]

# ************************************************
# HERE WE SELECT WHICH SIGNAL TO WORK WITH
signal = signal1
# ************************************************

In [ ]:
# 2 INTEGRATE A SIMPLE SIGNAL EXACTLY 
area = spi.trapezoid(signal, x)

# PLOT
plt.figure(figsize=(10, 6))
plt.annotate(f'area under signal: {area:.3f}', xy=(0.98, 0.98), xycoords='axes fraction', ha='right', va='top',color=colours['blue'])
plt.plot(x, signal, label='signal', color=colours['blue'])
plt.legend()
plt.xlabel('$x$')
plt.ylabel('signal')
plt.title('Exact Integration')

In [ ]:
# 3 COMPARE EXACT AND APPROXIMATE INTEGRATION OF A SIMPLE SIGNAL 

# INTEGRATE EXACTLY
area = spi.trapezoid(signal, x)

# INTEGRATE APPROXIMATELY
number_of_trapezoids = 5
area_approx, x_subset, y_subset = integrate_approx(signal, x, number_of_trapezoids)

# PLOT BOTH RESULTS
plt.figure(figsize=(10, 6))
plt.annotate(f'area under signal: {area:.3f}', xy=(0.98, 0.98), xycoords='axes fraction', color=colours['blue'], ha='right', va='top')
plt.annotate(f'approx area (n={number_of_trapezoids}): {area_approx:.3f}', xy=(0.98, 0.93), xycoords='axes fraction', color=colours['orange'], ha='right', va='top')
plt.plot(x, signal, label='signal', color=colours['blue'])
plt.plot(x_subset, y_subset, 'o-', color=colours['orange'], label='approx')
plt.legend()
plt.xlabel('$x$')
plt.ylabel('signal')
plt.title('Exact vs Approximate Integration')

In [ ]:
# 4 INTEGRATE A MORE COMPLEX SIGNAL EXACTLY 

# ************************************************
# LETS TRY ANOTHER SIGNAL
signal = signal3
# ************************************************

# INTEGRATE EXACTLY 
area = spi.trapezoid(signal, x)

# PLOT
plt.figure(figsize=(10, 6))
plt.annotate(f'area under signal: {area:.3f}', xy=(0.98, 0.98), xycoords='axes fraction', ha='right', va='top',color=colours['blue'])
plt.plot(x, signal, label='signal', color=colours['blue'])
plt.legend()
plt.xlabel('x')
plt.ylabel('signal')
plt.title('Exact Integration')

In [ ]:
# 5 COMPARE EXACT AND APPROXIMATE INTEGRATION OF A MORE COMPLEX SIGNAL 
# INTEGRATE EXACTLY
area = spi.trapezoid(signal, x)

# ************************************************
# INTEGRATE APPROXIMATELY
number_of_trapezoids = 5
area_approx, x_subset, y_subset = integrate_approx(signal, x, number_of_trapezoids)
# ************************************************

# PLOT BOTH RESULTS
plt.figure(figsize=(10, 6))
plt.annotate(f'area under signal: {area:.3f}', xy=(0.98, 0.98), xycoords='axes fraction', color=colours['blue'], ha='right', va='top')
plt.annotate(f'approx area (n={number_of_trapezoids}): {area_approx:.3f}', xy=(0.98, 0.93), xycoords='axes fraction', color=colours['orange'], ha='right', va='top')
plt.plot(x, signal, label='signal', color=colours['blue'])
plt.plot(x_subset, y_subset, 'o-', color=colours['orange'], label='approx')
plt.legend()
plt.xlabel('$x$')
plt.ylabel('signal')
plt.title('Exact vs Approximate Integration')

In [ ]:
# 6 DECONVOLUTE MANUALLY
# ************************************************
# DEFINE THE GUESS GAUSSIANS
amplitude = [2.0, 1.8] # peak heights
mu = [4, 5] # peak centers
sigma = [0.1, 0.1] # peak widths
# ************************************************

# CALCULATE THE GUESS GAUSSIANS
guess = [amplitude[0], mu[0], sigma[0], amplitude[1], mu[1], sigma[1]]
g1_guess = gauss(x, amplitude[0], mu[0], sigma[0])
g2_guess = gauss(x, amplitude[1], mu[1], sigma[1]) 
g_sum_guess = g1_guess + g2_guess

# PLOT THE GUESS GAUSSIANS
plt.figure(figsize=(10, 6))
plt.plot(x, signal, label='signal', color=colours['blue'])
plt.plot(x, g_sum_guess, label='sum fit', color=colours['grey'], linestyle='--')
plt.legend()
plt.xlabel('$x$')
plt.ylabel('signal')
plt.title('Manual Deconvolution - Guess')

In [ ]:
# 7 DECONVOLUTE AUTOMATICALLY
# DO THE FITTING 
popt, pcov = curve_fit(two_gauss, x, signal, p0=guess)
amplitude[0], mu[0], sigma[0], amplitude[1], mu[1], sigma[1] = popt

# CALCULATE THE FITTED GAUSSIANS
g1 = gauss(x, amplitude[0], mu[0], sigma[0])
g2 = gauss(x, amplitude[1], mu[1], sigma[1])
g_sum = g1 + g2

# CALCULATE THE AREA OF EACH GAUSSIAN ANALYTICALLY
area_g1 = amplitude[0] * sigma[0] * sqrt(2 * pi)
area_g2 = amplitude[1] * sigma[1] * sqrt(2 * pi)

# CALCULATE THE TOTAL AREA FROM THE FITTED GAUSSIANS
area_total = area_g1 + area_g2
# CALCULATE THE TOTAL AREA EXACTLY
area_exact = spi.trapezoid(signal, x)

# PLOT THE FIT
plt.figure(figsize=(10, 6))
plt.plot(x, g1, label=f'g1 ($A$ = {area_g1:.3f})', color=colours['green'])
plt.plot(x, g2, label=f'g2 ($A$ = {area_g2:.3f})', color=colours['purple'])
plt.plot(x, signal, label='signal', color=colours['blue'])
plt.plot(x, g_sum, label='sum fit', color=colours['grey'], linestyle='--')
plt.annotate(f'exact area: {area_exact:.3f}', xy=(0.98, 0.98), xycoords='axes fraction', color=colours['blue'], ha='right', va='top')
plt.annotate(f'sum of gaussians: {area_total:.3f}', xy=(0.98, 0.93), xycoords='axes fraction', color=colours['grey'], ha='right', va='top')
plt.legend(loc='upper left')
plt.xlabel('$x$')
plt.ylabel('signal')
plt.title('Automatic Deconvolution - Fit')

In [ ]:
# 8 KINETICS OF SIMPLE REACTIONS 
# DEFINE THE REACTIONS 
def first_order(t, y, k):
    dy_dt = -k * y
    title_text = f"d$[\\mathrm{{R}}]$/d$t$ = $-k [\\mathrm{{R}}]$"
    return dy_dt, title_text

def second_order(t, y, k):
    dy_dt = -2 * k * y**2
    title_text = f"d$[\\mathrm{{R}}]$/d$t$ = $-2 k [\\mathrm{{R}}]^2$"
    return dy_dt, title_text

def third_order(t, y, k):
    dy_dt = -3 * k * y**3
    title_text = f"d$[\\mathrm{{R}}]$/d$t$ = $-3 k [\\mathrm{{R}}]^3$"
    return dy_dt, title_text

odes = [first_order, second_order, third_order]

# ************************************************
# DEFINE STARTING CONDITIONS
t_start = 0.0
t_end = 5.0 # time range for simulation
t_number = 200  # number of time points
react_start = 1.0  # initial concentration of reactant
# ************************************************

# SOLVE NUMERICALLY AND PLOT
t = np.linspace(t_start, t_end, t_number)

def solve_and_plot(k = 1.0):
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    for ax, ode_f in zip(axes, odes):
        # get a title from the ODE (title_text does not depend on t/y here)
        _, title_text = ode_f(0, 1, k)
        
        def fun(t, y, k, ode_f=ode_f):
            dy, _ = ode_f(t, y, k)
            return dy
        
        sol = solve_ivp(fun, [t_start, t_end], [react_start], args=(k,), t_eval=t)
        ax.plot(sol.t, sol.y[0], color=colours['blue'], label='[R]')
        ax.plot(sol.t, react_start - sol.y[0], color=colours['orange'], label='[P]')
        ax.set_xlabel("$t$")
        ax.set_ylabel("$c(t)$")
        ax.set_title(title_text)
        ax.legend()
        ax.set_xlim(t_start, t_end)
        ax.set_ylim(0, react_start)
    
    plt.tight_layout()
    plt.show()

# Increase slider label and readout font size
display(HTML("<style>.widget-label, .widget-readout { font-size: 22px !important; }</style>"))
display(HTML("<style>.italic-k .widget-label { font-style: italic !important; }</style>"))
slider = FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1,
                     layout=Layout(width='500px', height='80px', margin='0 auto'))
slider.add_class('italic-k')

interact(solve_and_plot, k=slider);

In [ ]:
# 9 PARALLEL REACTIONS
def parallel(t, y, k1, k2):
    react, p1, p2 = y
    dreact_dt = -k1 * react - k2 * react
    dp1_dt = k1 * react
    dp2_dt = k2 * react
    return [dreact_dt, dp1_dt, dp2_dt]

def solve_and_plot_system(k1=0.1, k2=1.0):

    # ************************************************
    # DEFINE STARTING CONDITIONS
    t_start = 0.0
    t_end = 10.0 # time range for simulation
    t_number = 200  # number of time points
    react_start = 1.0  # initial concentration of reactant
    # ************************************************

    t = np.linspace(t_start, t_end, t_number)
    sol = solve_ivp(parallel, [t_start, t_end], [react_start, 0, 0], args=(k1, k2), t_eval=t)
    
    fig, ax = plt.subplots(figsize=(5, 4))
    
    # Plot react(t), p1, and p2
    ax.plot(sol.t, sol.y[0], label='$[\\mathrm{R}]$', color=colours['blue'])
    ax.plot(sol.t, sol.y[1], label='$[\\mathrm{P}_1]$', color=colours['yellow'])
    ax.plot(sol.t, sol.y[2], linestyle='dashdot', label='$[\\mathrm{P}_2]$', color=colours['purple'])

    ax.set_xlabel('$t$')
    ax.set_ylabel('$c(t)$')
    fig.suptitle(r"$\mathrm{P}_1 \leftarrow R \rightarrow \mathrm{P}_2$", y=0.90, x=0.58)
    ax.set_xlim(t_start, t_end)
    ax.set_ylim(0, react_start)
    ax.legend()
    plt.tight_layout()
    plt.show()
    
display(HTML("<style>.widget-label, .widget-readout { font-size: 22px !important; }</style>"))
display(HTML("<style>.italic-k .widget-label { font-style: italic !important; }</style>"))
k1_slider = FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1,
                        layout=Layout(width='500px', height='40px'))
k2_slider = FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1,
                        layout=Layout(width='500px', height='40px'))
k1_slider.add_class('italic-k')
k2_slider.add_class('italic-k')
k1_slider.description = 'k₁'
k2_slider.description = 'k₂'

interact(solve_and_plot_system, 
         k1=k1_slider,
         k2=k2_slider);

In [ ]:
# 10 CONSECUTIVE REACTIONS
def consecutive(t, y, k1, k2):
    react, intm = y
    dreact_dt = -k1 * react
    dintm_dt = k1 * react - k2 * intm
    return [dreact_dt, dintm_dt]

def solve_and_plot_system(k1=0.1, k2=1.0):

    # ************************************************
    # DEFINE STARTING CONDITIONS
    t_start = 0.0
    t_end = 10.0 # time range for simulation
    t_number = 200  # number of time points
    react_start = 1.0  # initial concentration of reactant
    # ************************************************

    t = np.linspace(t_start, t_end, t_number)
    sol = solve_ivp(consecutive, [t_start, t_end], [react_start, 0], args=(k1, k2), t_eval=t)
    
    fig, ax = plt.subplots(figsize=(5, 4))
    
    # Plot react(t) and intm(t)
    ax.plot(sol.t, sol.y[0], label='$[\\mathrm{R}]$', color=colours['blue'])
    ax.plot(sol.t, sol.y[1], label='$[\\mathrm{I}]$', color=colours['green'])
    ax.plot(sol.t, 1 - sol.y[0] - sol.y[1], label='$[\\mathrm{P}]$', color=colours['orange'])

    ax.set_xlabel('$t$')
    ax.set_ylabel('$c(t)$')
    fig.suptitle(r"R → I → P", y=0.90, x = 0.58)
    ax.legend()
    plt.tight_layout()
    plt.show()
    
display(HTML("<style>.widget-label, .widget-readout { font-size: 22px !important; }</style>"))
display(HTML("<style>.italic-k .widget-label { font-style: italic !important; }</style>"))
k1_slider = FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1,
                        layout=Layout(width='500px', height='40px'))
k2_slider = FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1,
                        layout=Layout(width='500px', height='40px'))
k1_slider.add_class('italic-k')
k2_slider.add_class('italic-k')
k1_slider.description = 'k₁'
k2_slider.description = 'k₂'

interact(solve_and_plot_system, 
         k1=k1_slider,
         k2=k2_slider);

In [ ]:
# 11 PRE-EQUILIBRIUM 
def pre_equilibrium(t, y, k1, km1, k2):
    react, intm = y
    dreact_dt = -k1 * react + km1 * intm
    dintm_dt = k1 * react - (km1 + k2) * intm
    return [dreact_dt, dintm_dt]

def solve_and_plot_system(k1=0.1, km1=0.1, k2=1.0):

    # ************************************************
    # DEFINE STARTING CONDITIONS
    t_start = 0.0
    t_end = 10.0 # time range for simulation
    t_number = 200  # number of time points
    react_start = 1.0  # initial concentration of reactant
    # ************************************************

    t = np.linspace(t_start, t_end, t_number)
    sol = solve_ivp(pre_equilibrium, [t_start, t_end], [react_start, 0], args=(k1, km1, k2), t_eval=t)
    
    fig, ax = plt.subplots(figsize=(5, 4))
    
    # Plot react(t), p1, and p2
    ax.plot(sol.t, sol.y[0], label='$[\\mathrm{R}]$', color=colours['blue'])
    ax.plot(sol.t, sol.y[1], label='$[\\mathrm{I}]$', color=colours['green'])
    ax.plot(sol.t, react_start - sol.y[0] - sol.y[1], label='$[\\mathrm{P}]$', color=colours['orange'])

    ax.set_xlabel('$t$')
    ax.set_ylabel('$c(t)$')
    fig.suptitle(r"R ⇆ I → P", y=0.90, x = 0.58)
    ax.legend()
    plt.tight_layout()
    plt.show()
    
display(HTML("<style>.widget-label, .widget-readout { font-size: 22px !important; }</style>"))
display(HTML("<style>.italic-k .widget-label { font-style: italic !important; }</style>"))
k1_slider = FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1,
                        layout=Layout(width='500px', height='40px'))
k2_slider = FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1,
                        layout=Layout(width='500px', height='40px'))
km1_slider = FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1,
                        layout=Layout(width='500px', height='40px'))

k1_slider.add_class('italic-k')
km1_slider.add_class('italic-k')
k2_slider.add_class('italic-k')
k1_slider.description = 'k₁'
km1_slider.description = 'k₋₁'
k2_slider.description = 'k₂'

interact(solve_and_plot_system, 
         k1=k1_slider,
         km1=km1_slider,
         k2=k2_slider);